# Nested Attributes & Functions Operating on Nested Types in PySpark

In this notebook we will be working with spotify songs Dataset from Kaggle. Specifically we will work with nested data types where the columns are to type arrays or maps.

In [1]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 400)

In [3]:
# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=rnd_seed
np.random.set_state=rnd_seed

## 1. Create the Spark Session

In [4]:
spark = SparkSession.builder.master("local[*]").appName("nested_attributes").getOrCreate()

In [5]:
spark

In [6]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=nested_attributes>

In [7]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

In [8]:
import re

# Utility function to emulate stripMargin in Scala string.
def strip_margin(text):
    nomargin = re.sub('\n[ \t]*\|', ' ', text)
    trimmed = re.sub('\s+', ' ', nomargin)
    return trimmed

## 2. Load Spotify Songs Dataset

In [9]:
spotify_df = spark.read.csv(path='data/spotify-songs.csv', inferSchema=True, header=True).cache()

In [10]:
spotify_df.limit(10).toPandas()

,id,song_title,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
0,0,Mask Off,Future,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4,0.286,1
1,1,Redbone,Childish Gambino,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4,0.588,1
2,2,Xanny Family,Future,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4,0.173,1
3,3,Master Of None,Beach House,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4,0.230,1
4,4,Parallel Lines,Junior Boys,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4,0.904,1
5,5,Sneakin’,Drake,0.00479,0.804,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4,0.264,1
6,6,Childs Play,Drake,0.01450,0.739,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4,0.308,1
7,7,Gyöngyhajú lány,Omega,0.02020,0.266,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4,0.393,1
8,8,I've Seen Footage,Death Grips,0.04810,0.603,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4,0.398,1
9,9,Digital Animal,Honey Claws,0.00208,0.836,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4,0.386,1


## 3. Data Wrangling

### 3.1 Create Nested Types

+ Combine the columns ['key', 'mode', 'target'] into an array using the `array` function of PySpark. 
+ Transform the acoustic qualities {'acousticness', 'tempo', 'liveness', 'instrumentalness', 'energy', 'danceability', 'speechiness', 'loudness'} of a song from individual columns into a map (key being acoustic quality). Although `create_map` function is meant to create map between a pair of columns but here we use the F.lit(...) function to generate the string key name for the acoustic quality.
http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark.sql.functions.create_map

In [11]:
map_df = (spotify_df
          .select('id', 'song_title', 'artist', 'duration_ms',
                  F.array('key', 'mode', 'target').alias('audience'), 
                  F.create_map(
                      F.lit('acousticness'), 'acousticness', 
                      F.lit('danceability'), 'acousticness',
                      F.lit('energy'), 'energy',
                      F.lit('instrumentalness'), 'instrumentalness',
                      F.lit('liveness'), 'liveness',
                      F.lit('loudness'), 'loudness',
                      F.lit('speechiness'), 'speechiness',
                      F.lit('tempo'), 'tempo'
                  ).alias('qualities'),
                 'time_signature',
                 'valence')
        .cache())

In [12]:
map_df.limit(10).toPandas()

,id,song_title,artist,duration_ms,audience,qualities,time_signature,valence
0,0,Mask Off,Future,204600,"[2, 1, 1]","{'acousticness': 0.0102, 'loudness': -8.795, 'liveness': 0.165, 'tempo': 150.062, 'instrumentalness': 0.0219, 'danceability': 0.0102, 'speechiness': 0.431, 'energy': 0.434}",4,0.286
1,1,Redbone,Childish Gambino,326933,"[1, 1, 1]","{'acousticness': 0.199, 'loudness': -10.401, 'liveness': 0.137, 'tempo': 160.083, 'instrumentalness': 0.00611, 'danceability': 0.199, 'speechiness': 0.0794, 'energy': 0.359}",4,0.588
2,2,Xanny Family,Future,185707,"[2, 1, 1]","{'acousticness': 0.0344, 'loudness': -7.148, 'liveness': 0.159, 'tempo': 75.044, 'instrumentalness': 0.000234, 'danceability': 0.0344, 'speechiness': 0.289, 'energy': 0.412}",4,0.173
3,3,Master Of None,Beach House,199413,"[5, 1, 1]","{'acousticness': 0.604, 'loudness': -15.236, 'liveness': 0.0922, 'tempo': 86.468, 'instrumentalness': 0.51, 'danceability': 0.604, 'speechiness': 0.0261, 'energy': 0.338}",4,0.230
4,4,Parallel Lines,Junior Boys,392893,"[5, 0, 1]","{'acousticness': 0.18, 'loudness': -11.648, 'liveness': 0.439, 'tempo': 174.004, 'instrumentalness': 0.512, 'danceability': 0.18, 'speechiness': 0.0694, 'energy': 0.561}",4,0.904
5,5,Sneakin’,Drake,251333,"[8, 1, 1]","{'acousticness': 0.00479, 'loudness': -6.682, 'liveness': 0.164, 'tempo': 85.023, 'instrumentalness': 0.0, 'danceability': 0.00479, 'speechiness': 0.185, 'energy': 0.56}",4,0.264
6,6,Childs Play,Drake,241400,"[1, 1, 1]","{'acousticness': 0.0145, 'loudness': -11.204, 'liveness': 0.207, 'tempo': 80.03, 'instrumentalness': 7.27e-06, 'danceability': 0.0145, 'speechiness': 0.156, 'energy': 0.472}",4,0.308
7,7,Gyöngyhajú lány,Omega,349667,"[10, 0, 1]","{'acousticness': 0.0202, 'loudness': -11.609, 'liveness': 0.16, 'tempo': 144.154, 'instrumentalness': 0.664, 'danceability': 0.0202, 'speechiness': 0.0371, 'energy': 0.348}",4,0.393
8,8,I've Seen Footage,Death Grips,202853,"[11, 0, 1]","{'acousticness': 0.0481, 'loudness': -3.626, 'liveness': 0.342, 'tempo': 130.035, 'instrumentalness': 0.0, 'danceability': 0.0481, 'speechiness': 0.347, 'energy': 0.944}",4,0.398
9,9,Digital Animal,Honey Claws,226840,"[7, 1, 1]","{'acousticness': 0.00208, 'loudness': -7.792, 'liveness': 0.571, 'tempo': 99.994, 'instrumentalness': 0.0, 'danceability': 0.00208, 'speechiness': 0.237, 'energy': 0.603}",4,0.386


In [13]:
# Let's check the schema of the new DataFrame
map_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- song_title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- audience: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- qualities: map (nullable = false)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- time_signature: integer (nullable = true)
 |-- valence: double (nullable = true)



**Write the DataFrame to a json file:**

In [14]:
map_df.coalesce(1).write.json(path='data/spotify-songs', mode="overwrite")

### 3.2 Reload the above restructured DataFrame now using a more complex schema with Nested Data Types

In [15]:
nested_schema = StructType([
    StructField('id', IntegerType(), nullable=False),
    StructField('song_title', StringType(), nullable=False),
    StructField('artist', StringType(), nullable=False),
    StructField('duration_ms', IntegerType(), nullable=False),
    StructField('audience', ArrayType(elementType=IntegerType()), nullable=False),
    StructField('qualities', MapType(keyType=StringType(), valueType=DoubleType(), valueContainsNull=False), nullable=True),
    StructField('time_signature', IntegerType(), nullable=False),
    StructField('valence', DoubleType(), nullable=False),
  ])

In [16]:
spotify_reloaded_df = spark.read.json(path='data/spotify-songs', schema=nested_schema).cache()

In [17]:
spotify_reloaded_df.limit(10).toPandas()

,id,song_title,artist,duration_ms,audience,qualities,time_signature,valence
0,0,Mask Off,Future,204600,"[2, 1, 1]","{'acousticness': 0.0102, 'loudness': -8.795, 'liveness': 0.165, 'tempo': 150.062, 'instrumentalness': 0.0219, 'danceability': 0.0102, 'speechiness': 0.431, 'energy': 0.434}",4,0.286
1,1,Redbone,Childish Gambino,326933,"[1, 1, 1]","{'acousticness': 0.199, 'loudness': -10.401, 'liveness': 0.137, 'tempo': 160.083, 'instrumentalness': 0.00611, 'danceability': 0.199, 'speechiness': 0.0794, 'energy': 0.359}",4,0.588
2,2,Xanny Family,Future,185707,"[2, 1, 1]","{'acousticness': 0.0344, 'loudness': -7.148, 'liveness': 0.159, 'tempo': 75.044, 'instrumentalness': 0.000234, 'danceability': 0.0344, 'speechiness': 0.289, 'energy': 0.412}",4,0.173
3,3,Master Of None,Beach House,199413,"[5, 1, 1]","{'acousticness': 0.604, 'loudness': -15.236, 'liveness': 0.0922, 'tempo': 86.468, 'instrumentalness': 0.51, 'danceability': 0.604, 'speechiness': 0.0261, 'energy': 0.338}",4,0.230
4,4,Parallel Lines,Junior Boys,392893,"[5, 0, 1]","{'acousticness': 0.18, 'loudness': -11.648, 'liveness': 0.439, 'tempo': 174.004, 'instrumentalness': 0.512, 'danceability': 0.18, 'speechiness': 0.0694, 'energy': 0.561}",4,0.904
5,5,Sneakin’,Drake,251333,"[8, 1, 1]","{'acousticness': 0.00479, 'loudness': -6.682, 'liveness': 0.164, 'tempo': 85.023, 'instrumentalness': 0.0, 'danceability': 0.00479, 'speechiness': 0.185, 'energy': 0.56}",4,0.264
6,6,Childs Play,Drake,241400,"[1, 1, 1]","{'acousticness': 0.0145, 'loudness': -11.204, 'liveness': 0.207, 'tempo': 80.03, 'instrumentalness': 7.27e-06, 'danceability': 0.0145, 'speechiness': 0.156, 'energy': 0.472}",4,0.308
7,7,Gyöngyhajú lány,Omega,349667,"[10, 0, 1]","{'acousticness': 0.0202, 'loudness': -11.609, 'liveness': 0.16, 'tempo': 144.154, 'instrumentalness': 0.664, 'danceability': 0.0202, 'speechiness': 0.0371, 'energy': 0.348}",4,0.393
8,8,I've Seen Footage,Death Grips,202853,"[11, 0, 1]","{'acousticness': 0.0481, 'loudness': -3.626, 'liveness': 0.342, 'tempo': 130.035, 'instrumentalness': 0.0, 'danceability': 0.0481, 'speechiness': 0.347, 'energy': 0.944}",4,0.398
9,9,Digital Animal,Honey Claws,226840,"[7, 1, 1]","{'acousticness': 0.00208, 'loudness': -7.792, 'liveness': 0.571, 'tempo': 99.994, 'instrumentalness': 0.0, 'danceability': 0.00208, 'speechiness': 0.237, 'energy': 0.603}",4,0.386


In [18]:
spotify_reloaded_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- song_title: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- audience: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- qualities: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- time_signature: integer (nullable = true)
 |-- valence: double (nullable = true)



### 3.3 Extract Individual Nested/Complex Atributes as a Column

We can extract out each nested attribute within an array or map into a column of its own.

**Extract out array elements:**  
The audience column is a combination of three attributes 'key', 'mode' and 'target'. Extract out each array element into a column of its own.

In [19]:
(spotify_reloaded_df
 .select(col('audience').getItem(0).alias('key'), 
         col('audience').getItem(1).alias('mode'),
         col('audience').getItem(2).alias('target'))
 .limit(10)
 .toPandas())

,key,mode,target
0,2,1,1
1,1,1,1
2,2,1,1
3,5,1,1
4,5,0,1
5,8,1,1
6,1,1,1
7,10,0,1
8,11,0,1
9,7,1,1


**Extract out Map attributes:**  
The acoustic column is a map created from attributes 'acousticness', 'tempo', 'liveness', 'instrumentalness', etc. of a song. Extract out those qualities into individual columns.

In [20]:
(spotify_reloaded_df
 .select(
     col('qualities').getItem('acousticness').alias('acousticness'),
     col('qualities').getItem('speechiness').alias('speechiness')
 )
 .limit(10)
 .toPandas())

,acousticness,speechiness
0,0.01020,0.4310
1,0.19900,0.0794
2,0.03440,0.2890
3,0.60400,0.0261
4,0.18000,0.0694
5,0.00479,0.1850
6,0.01450,0.1560
7,0.02020,0.0371
8,0.04810,0.3470
9,0.00208,0.2370


**Reconstruct the original Table:**

In [21]:
(spotify_reloaded_df
 .select('id', 'song_title', 'artist',
     col('qualities').getItem('acousticness').alias('acousticness'),
     col('qualities').getItem('danceability').alias('danceability'),
     'duration_ms',
     col('qualities').getItem('energy').alias('energy'),
     col('qualities').getItem('instrumentalness').alias('instrumentalness'),
     col('audience').getItem(0).alias('key'),
     col('qualities').getItem('liveness').alias('liveness'),
     col('qualities').getItem('loudness').alias('loudness'),
     col('audience').getItem(1).alias('mode'),
     col('qualities').getItem('speechiness').alias('speechiness'),
     col('qualities').getItem('tempo').alias('tempo'),
     'time_signature',
     'valence',
     col('audience').getItem(2).alias('target')
 )
 .limit(10)
 .toPandas())

,id,song_title,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target
0,0,Mask Off,Future,0.01020,0.01020,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4,0.286,1
1,1,Redbone,Childish Gambino,0.19900,0.19900,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4,0.588,1
2,2,Xanny Family,Future,0.03440,0.03440,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4,0.173,1
3,3,Master Of None,Beach House,0.60400,0.60400,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4,0.230,1
4,4,Parallel Lines,Junior Boys,0.18000,0.18000,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4,0.904,1
5,5,Sneakin’,Drake,0.00479,0.00479,251333,0.560,0.000000,8,0.1640,-6.682,1,0.1850,85.023,4,0.264,1
6,6,Childs Play,Drake,0.01450,0.01450,241400,0.472,0.000007,1,0.2070,-11.204,1,0.1560,80.030,4,0.308,1
7,7,Gyöngyhajú lány,Omega,0.02020,0.02020,349667,0.348,0.664000,10,0.1600,-11.609,0,0.0371,144.154,4,0.393,1
8,8,I've Seen Footage,Death Grips,0.04810,0.04810,202853,0.944,0.000000,11,0.3420,-3.626,0,0.3470,130.035,4,0.398,1
9,9,Digital Animal,Honey Claws,0.00208,0.00208,226840,0.603,0.000000,7,0.5710,-7.792,1,0.2370,99.994,4,0.386,1


### 3.4 Explode Individual Nested/Complex into a row of its own

Using `posexplode` function we can extract array element into a new row for each element with position in the given array.

In [22]:
(spotify_reloaded_df
 .select(F.posexplode('audience'))
 .limit(10)
 .toPandas())

,pos,col
0,0,2
1,1,1
2,2,1
3,0,1
4,1,1
5,2,1
6,0,2
7,1,1
8,2,1
9,0,5


Using `explode` function we can extract a new row for each element in the given array or map.

In [23]:
(spotify_reloaded_df
 .select(F.explode('qualities').alias("qualities", "value"))
 .limit(10)
 .toPandas())

,qualities,value
0,acousticness,0.0102
1,danceability,0.0102
2,energy,0.4340
3,instrumentalness,0.0219
4,liveness,0.1650
5,loudness,-8.7950
6,speechiness,0.4310
7,tempo,150.0620
8,acousticness,0.1990
9,danceability,0.1990


In [24]:
spark.stop()